## Загрузим нужные библиотеки

In [1]:
!pip install catboost
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import uniform
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import BorderlineSMOTE 
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTENC
from sklearn import svm
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf 
import numpy as np
import pickle
from sklearn.naive_bayes import MultinomialNB
import catboost as cb
%matplotlib inline

2022-11-10 19:53:41.711161: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 19:53:41.850880: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 19:53:41.850903: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-10 19:53:41.888208: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-10 19:53:42.569045: W tensorflow/stream_executor/platform/de

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("train_frame.csv",low_memory=False)
df

,id,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
0,6818780,26,17235,18,1,1,0.0,2,0,87.0,5.0,1.0,0.0,2.0,0.00,0.0,46654,41.0,150.0,42.0,720176.0,58950.0,779126.0,8290896.0,0,0,0,0,0
1,9907176,20,23,4,1,1,0.0,1,0,107.0,5.0,1.0,0.0,2.0,0.00,0.0,194484,68.0,400.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0,0
2,3304275,15,16331,19,1,1,0.0,2,0,50.0,5.0,1.0,0.0,2.0,0.00,0.0,92224,56.0,218.0,62.0,3246292.0,3233068.0,6479360.0,52708071.0,0,1,0,0,0
3,9020937,16,6325,19,1,1,0.0,2,0,416.0,5.0,1.0,0.0,2.0,35.34,0.0,79671,33.0,100.0,55.0,2060928.0,653280.0,2714208.0,19562334.0,0,0,0,0,0
4,3082311,17,17315,18,1,1,0.0,2,0,795.0,5.0,1.0,0.0,2.0,52.52,0.0,141303,716.0,1000.0,16.0,316919.0,27911.0,344830.0,4719186.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5999995,9958614,19,23,4,1,1,0.0,1,0,25.0,5.0,0.0,0.0,2.0,0.00,0.0,63431,24.0,100.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,1,0,0
5999996,2234489,19,4697,19,1,1,0.0,2,0,83.0,5.0,1.0,0.0,2.0,0.00,0.0,36730,100.0,1832.0,31.0,1767370.0,144063.0,1911433.0,15582018.0,0,0,0,0,0
5999997,4304572,24,18,19,3,1,0.0,1,0,1700.0,5.0,1.0,0.0,2.0,94.09,0.0,37295,952.0,800.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,0,0,0,0
5999998,6550634,15,6,19,3,1,0.0,1,0,269.0,5.0,1.0,0.0,2.0,0.00,0.0,101,0.0,0.0,105.0,15091338.0,4972424.0,20063762.0,39988530.0,0,1,0,0,0


Обьединим список не нужных строк с списком строк типа object

In [4]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [5]:
col_obj

['label', 'id']

In [6]:
X = df.drop(col_obj, axis = 1)
y = df["label"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)
X_train

,oper_type + oper_attr,index_oper,type,priority,is_privatecategory,class,is_in_yandex,is_return,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,name_mfi,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address
3877479,93,16,15,1,1,0.0,1,0,435.0,5.0,1.0,0.0,2.0,0.00,0.0,72298,400.0,1800.0,40.0,9560324.0,2169758.0,11730082.0,11979988.0,0,0,0,0
5032826,19,127,19,3,1,0.0,1,0,1307.0,5.0,1.0,0.0,2.0,0.00,0.0,8406,196.0,4716.0,137.0,20699228.0,49882.0,20749110.0,218661166.0,0,0,0,0
5070446,17,20786,13,2,1,3.0,2,0,657.0,5.0,1.0,0.0,2.0,0.00,0.0,101,0.0,0.0,2.0,1521.0,846.0,2367.0,24809.0,0,1,0,0
4482246,17,16339,18,1,1,0.0,1,0,13.0,5.0,0.0,0.0,2.0,0.00,0.0,104192,20.0,100.0,16.0,456104.0,14495806.0,14951910.0,15022023.0,0,0,0,0
1398411,17,18,19,3,1,0.0,1,0,10.0,5.0,0.0,0.0,2.0,0.00,0.0,119179,10.0,100.0,186.0,60613352.0,10648.0,60624000.0,75592387.0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333373,16,15670,3,1,1,3.0,2,0,434.0,5.0,1.0,0.0,2.0,0.00,0.0,60585,300.0,9300.0,17.0,13933.0,5720.0,19653.0,274188.0,0,0,0,0
3666771,58,23,4,1,1,0.0,1,0,41.0,5.0,1.0,0.0,2.0,0.00,0.0,46934,19.0,50.0,914.0,48856658.0,83318932.0,132175590.0,136819803.0,0,0,0,0
5484193,9,23,4,1,1,0.0,1,0,475.0,5.0,1.0,0.0,2.0,38.11,0.0,2611,86.0,133.0,1089.0,64270133.0,116432632.0,180702765.0,188407812.0,0,0,0,0
196076,17,9746,18,1,1,0.0,2,0,302.0,5.0,1.0,0.0,2.0,33.01,0.0,101,0.0,0.0,20.0,1113748.0,351845.0,1465593.0,9888343.0,0,1,0,0


In [8]:
sm = SMOTE(random_state=42,n_jobs=10)

X_train, y_train = sm.fit_resample(X_train, y_train)

In [9]:
X_train.shape

(11541490, 27)

## Обучение модели

In [10]:
scaler = MinMaxScaler()
# logistic = OneVsRestClassifier(RandomForestClassifier(max_depth=22,n_jobs=-1))
#logistic=GaussianNB()
#pipe = Pipeline([('normalizer', scaler), ('classifier', logistic)])
#pipe.fit(X_train, y_train)

clf1=HistGradientBoostingClassifier(learning_rate=0.1,max_depth=8, min_samples_leaf=100)
clf2=RandomForestClassifier(max_depth=15,n_jobs=9)
clf3=GaussianNB()
eclf1 = VotingClassifier([('gb', clf1),('rf', clf2),('gnb', clf3)], voting='soft',weights=[1,1.1,1.2])
pipe = Pipeline([('normalizer', scaler), ('classifier', eclf1)])
pipe.fit(X_train, y_train)

predictions = pipe.predict(X_test)
print(classification_report(y_test, predictions))

pool_train = cb.Pool(X, y)
params = {                  # 'task_type':'GPU',
                            'iterations':1200,
                            'depth':8,
                            'random_state':2,
                            'learning_rate':0.01,
                            'eval_metric':'Recall',
                            'loss_function':'MultiCrossEntropy'}
clf_with_aux = cb.CatBoostClassifier(**params)
clf_with_aux.fit(pool_train)


predcat = clf_with_aux.predict_proba(X_test)
voting = pipe.predict_proba(X_test)


y_pred = ((voting + predcat) / 2).argmax(axis=1)


print(classification_report(y_test, y_pred))



parametrs = {
              'max_depth': range (1,35),
            }
grid = GridSearchCV(RandomForestClassifier(), parametrs, scoring='roc_auc', cv=2,n_jobs=9) 
grid.fit(X_test, y_test) 
grid.best_params_



pkl_filename = "model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(pipe, file)

with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)

nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB, 
                 cv=2,   # use any cross validation technique 
                 verbose=1, 
                 scoring='roc_auc') 
gs_NB.fit(X_train, y_train)

gs_NB.best_params_


gb_grid_params = {'learning_rate': [0.1, 0.05, 0.02, 0.01],
              'max_depth': [4, 6, 8],
              'min_samples_leaf': [20, 50,100,150],
              #'max_features': [1.0, 0.3, 0.1] 
              }
print(gb_grid_params)

gb_gs = HistGradientBoostingClassifier()

clf = GridSearchCV(gb_gs,
                               gb_grid_params,
                               cv=2,
                               scoring='roc_auc',
                               verbose = 3, 
                               n_jobs=2);
clf.fit(X_train, y_train);
clf.best_params_

# Нейронка

In [11]:
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [12]:
from keras import models
from keras import layers
import tensorflow as tf 
def build_model1():
    model = models.Sequential()
    model.add(layers.Dense(270,activation='sigmoid',input_shape=(27,)))
    model.add(layers.Dense(9,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',# Вы также можете указать параметры для оптимизатора через optimizer = optimizer.RMSprop (lr = 0.001)
                  loss='binary_crossentropy', # Эквивалент потерь = loss.binary_crossentropy
                  metrics=[tf.keras.metrics.Recall()]) # Эквивалентно метрике = [metircs.binary_accuracy]
    return model
model1 = build_model1()


history = model1.fit(X_train, y_train,
                    epochs=5, # Итерировать 20 раз по полному набору данных
                    batch_size=512, # Размер каждой партии 512
                    validation_data=(X_test,y_test))

2022-11-10 19:54:32.502323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 19:54:32.503586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-10 19:54:32.503780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-10 19:54:32.503938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-10 19:54:32.504098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/5
22542/22542 [==============================] - 58s 3ms/step - loss: 0.1742 - recall: 0.9724 - val_loss: 0.4406 - val_recall: 0.8099
Epoch 2/5
22542/22542 [==============================] - 57s 3ms/step - loss: 0.1175 - recall: 0.9782 - val_loss: 1.1643 - val_recall: 0.7455
Epoch 3/5
22542/22542 [==============================] - 57s 3ms/step - loss: 0.0976 - recall: 0.9869 - val_loss: 2.1289 - val_recall: 0.7536
Epoch 4/5
22542/22542 [==============================] - 56s 2ms/step - loss: 0.0900 - recall: 0.9935 - val_loss: 2.6846 - val_recall: 0.7438
Epoch 5/5
22542/22542 [==============================] - 56s 2ms/step - loss: 0.0862 - recall: 0.9940 - val_loss: 2.5775 - val_recall: 0.7194


In [13]:
predictions = model1.predict(X_test)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
print('нейронка',classification_report(y_test, pred))

1875/1875 [==============================] - 2s 835us/step
нейронка               precision    recall  f1-score   support

           0       0.99      0.77      0.87     58275
           1       0.09      0.72      0.15      1725

    accuracy                           0.77     60000
   macro avg       0.54      0.75      0.51     60000
weighted avg       0.96      0.77      0.85     60000



# Нейронка 2


In [32]:
from keras import models
from keras import layers
import tensorflow as tf 
def build_model2():
    model = models.Sequential()
    model.add(layers.Dense(270,activation='sigmoid',input_shape=(27,)))
    model.add(layers.Dense(90,activation='relu'))
    model.add(layers.Dense(30,activation='relu'))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',# Вы также можете указать параметры для оптимизатора через optimizer = optimizer.RMSprop (lr = 0.001)
                  loss='binary_crossentropy', # Эквивалент потерь = loss.binary_crossentropy
                  metrics=[tf.keras.metrics.Recall()]) # Эквивалентно метрике = [metircs.binary_accuracy]
    return model
model2 = build_model2()


history = model2.fit(X_train, y_train,
                    epochs=1, # Итерировать 20 раз по полному набору данных
                    batch_size=512, # Размер каждой партии 512
                    validation_data=(X_test,y_test))

22542/22542 [==============================] - 86s 4ms/step - loss: 0.1167 - recall_3: 0.9861 - val_loss: 0.2482 - val_recall_3: 0.7501


In [33]:
predictions = model2.predict(X_test)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
print('нейронка',classification_report(y_test, pred))

1875/1875 [==============================] - 2s 988us/step
нейронка               precision    recall  f1-score   support

           0       0.99      0.92      0.95     58275
           1       0.21      0.75      0.33      1725

    accuracy                           0.91     60000
   macro avg       0.60      0.83      0.64     60000
weighted avg       0.97      0.91      0.93     60000



In [16]:
df_nero = pd.DataFrame() 
df_nero['1']= model1.predict(X_test).flatten().tolist ()

df_nero['2']= model2.predict(X_test).flatten().tolist ()

df_nero['pre']=(df_nero['1']+df_nero['2'])/2

pred=[]
for i in  df_nero['pre']:
    pred.append(round(float(i)))

print('нейронка',classification_report(y_test, pred))

1875/1875 [==============================] - 2s 894us/step
нейронка               precision    recall  f1-score   support

           0       0.99      0.88      0.93     58275
           1       0.16      0.73      0.26      1725

    accuracy                           0.88     60000
   macro avg       0.57      0.81      0.59     60000
weighted avg       0.97      0.88      0.91     60000



## Оценка точности

In [17]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [18]:
print("Recall", score)

Recall 0.8077458825284912


In [19]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.99      0.88      0.93     58275
           1       0.16      0.73      0.26      1725

    accuracy                           0.88     60000
   macro avg       0.57      0.81      0.59     60000
weighted avg       0.97      0.88      0.91     60000



# Предсказание 

In [20]:
pred_data=pd.read_csv('test_dataset_test.csv',low_memory=False)


In [21]:
df=pred_data

In [22]:
df["priority"] = pd.Categorical(df["priority"])
df["priority"].astype('category').cat.codes
df["priority"] = df["priority"].cat.codes
df["is_in_yandex"] = pd.Categorical(df["is_in_yandex"])
df["is_in_yandex"].astype('category').cat.codes
df["is_in_yandex"] = df["is_in_yandex"].cat.codes
df["is_return"] = pd.Categorical(df["is_return"])
df["is_return"].astype('category').cat.codes
df["is_return"] = df["is_return"].cat.codes
df["oper_type + oper_attr"] = pd.Categorical(df["oper_type + oper_attr"])
df["oper_type + oper_attr"].astype('category').cat.codes
df["oper_type + oper_attr"] = df["oper_type + oper_attr"].cat.codes
df["index_oper"] = pd.Categorical(df["index_oper"])
df["index_oper"].astype('category').cat.codes
df["index_oper"] = df["index_oper"].cat.codes
df["type"] = pd.Categorical(df["type"])
df["type"].astype('category').cat.codes
df["type"] = df["type"].cat.codes
df["is_privatecategory"] = pd.Categorical(df["is_privatecategory"])
df["is_privatecategory"].astype('category').cat.codes
df["is_privatecategory"] = df["is_privatecategory"].cat.codes
df["name_mfi"] = pd.Categorical(df["name_mfi"])
df["name_mfi"].astype('category').cat.codes
df["name_mfi"] = df["name_mfi"].cat.codes

In [23]:
df = df.fillna(0)


In [24]:
df=df.drop(['id'], axis = 1)

In [25]:
pred_data['label']=pipe.predict(df)

NameError: name 'pipe' is not defined

In [ ]:
pred_data[['id','label']].to_csv('bestmodel.csv', index=False)

# Нейро 

In [34]:
df=scaler.transform(df)
predictions = model2.predict(df)
pred=[]
for i in predictions:
    pred.append(round(float(i)))
pred_data['label']=pred
pred_data[['id','label']].to_csv('bestmodelnero.csv', index=False)

/home/gennalll/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


 21912/125000 [====>.........................] - ETA: 1:36

KeyboardInterrupt: 

# Нейро 2

In [26]:
df_nero = pd.DataFrame() 
df_nero['1']= model1.predict(df).flatten().tolist ()

df_nero['2']= model2.predict(df).flatten().tolist ()

df_nero['pre']=(df_nero['1']+df_nero['2'])/2

pred=[]
for i in  df_nero['pre']:
    pred.append(round(float(i)))
pred_data['label']=pred
pred_data[['id','label']].to_csv('bestmodelnero2.csv', index=False)


125000/125000 [==============================] - 116s 928us/step


In [28]:
pred

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
